In [1]:
import sqlite3 as sql
import pandas as pd

connection = sql.connect('sqlite_db_pythonsqlite.db')
cursor = connection.cursor()

In [2]:
# Q1: Some of the facilities charge a fee to members, but some do not.
#Write a SQL query to produce a list of the names of the facilities that do.

q1 = '''
SELECT name 
FROM Facilities 
WHERE membercost > 0;
'''
cursor.execute(q1)

print("{0:10s}".format('name'))
for res in cursor.fetchall():
    print("{0:10s}".format(*res))


name      
Tennis Court 1
Tennis Court 2
Massage Room 1
Massage Room 2
Squash Court


In [3]:
# Q2: How many facilities do not charge a fee to members?

q2 = '''
SELECT COUNT(name) AS Count 
FROM Facilities 
WHERE membercost = 0;
'''

cursor.execute(q2)

print("{0:10s}".format('Count'))
for res in cursor.fetchall():
    print("{0:0d}".format(*res))


Count     
4


In [4]:
# Q3: Write an SQL query to show a list of facilities that charge a fee to members,
# where the fee is less than 20% of the facility's monthly maintenance cost.
# Return the facid, facility name, member cost, and monthly maintenance of the
# facilities in question.

q3 = '''
SELECT facid, name, membercost, monthlymaintenance 
FROM Facilities 
WHERE membercost < (monthlymaintenance * .2) AND membercost > 0;
'''

cursor.execute(q3)

print("{0:1s} \t {1:15s} \t {2:10s} \t {3:10s}".format('facid', 'name', 'membercost', 'monthlymaintenance'))
for res in cursor.fetchall():
    print("{0:1d} \t {1:15s} \t {2:10.1f} \t {3:10d}".format(*res))


facid 	 name            	 membercost 	 monthlymaintenance
0 	 Tennis Court 1  	        5.0 	        200
1 	 Tennis Court 2  	        5.0 	        200
4 	 Massage Room 1  	        9.9 	       3000
5 	 Massage Room 2  	        9.9 	       3000
6 	 Squash Court    	        3.5 	         80


In [5]:
# Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
# Try writing the query without using the OR operator.

q4 = '''
SELECT * 
FROM Facilities 
WHERE facid IN (1,5);
'''

cursor.execute(q4)

print("{0:1s} \t {1:15s} \t {2:10s} \t {3:10s} \t {4:10s} \t {5:10s}".format('facid', 'name', 'membercost', 'guestcost', 'initialoutlay', 'monthlymaintenance'))
for res in cursor.fetchall():
    print("{0:1d} \t {1:10s} \t {2:10.1f} \t {3:10.1f} \t {4:10d} \t {5:10d}".format(*res))

facid 	 name            	 membercost 	 guestcost  	 initialoutlay 	 monthlymaintenance
1 	 Tennis Court 2 	        5.0 	       25.0 	       8000 	        200
5 	 Massage Room 2 	        9.9 	       80.0 	       4000 	       3000


In [6]:
# Q5: Produce a list of facilities, with each labelled as
#'cheap' or 'expensive', depending on if their monthly maintenance cost is
#more than $100. Return the name and monthly maintenance of the facilities
#in question.

q5 = '''
SELECT name, monthlymaintenance, CASE WHEN monthlymaintenance < 100 THEN 'cheap' ELSE 'expensive' END AS cost 
FROM Facilities;
'''

cursor.execute(q5)

print("{0:15s} \t {1:15s} \t {2:10s}".format('name', 'monthlymaintenance', 'cost'))
for res in cursor.fetchall():
    print("{0:15s} \t {1:15d} \t {2:10s}".format(*res))

name            	 monthlymaintenance 	 cost      
Tennis Court 1  	             200 	 expensive 
Tennis Court 2  	             200 	 expensive 
Badminton Court 	              50 	 cheap     
Table Tennis    	              10 	 cheap     
Massage Room 1  	            3000 	 expensive 
Massage Room 2  	            3000 	 expensive 
Squash Court    	              80 	 cheap     
Snooker Table   	              15 	 cheap     
Pool Table      	              15 	 cheap     


In [7]:
# Q6: You'd like to get the first and last name of the last member(s)
# who signed up. Try not to use the LIMIT clause for your solution.

q6 = '''
SELECT firstname, surname 
FROM Members 
WHERE joindate IN (SELECT MAX(joindate) FROM Members);
'''

cursor.execute(q6)

print("{0:10s} \t {1:10s}".format('firstname', 'surname'))
for res in cursor.fetchall():
    print("{0:10s} \t {1:10s}".format(*res))

firstname  	 surname   
Darren     	 Smith     


In [8]:
# Q7: Produce a list of all members who have used a tennis court.
#Include in your output the name of the court, and the name of the member
#formatted as a single column. Ensure no duplicate data, and order by
#the member name.

q7 = '''
SELECT DISTINCT CONCAT_WS(' ', m.firstname, m.surname) as Name, f.name as Court
FROM Bookings b
LEFT JOIN Facilities f USING (facid)
LEFT JOIN Members m USING (memid)
WHERE b.facid IN (0, 1) AND b.memid > 0
ORDER BY Name;
'''

In [9]:
# Q8: Produce a list of bookings on the day of 2012-09-14 which
#will cost the member (or guest) more than $30. Remember that guests have
#different costs to members (the listed costs are per half-hour 'slot'), and
#the guest user's ID is always 0. Include in your output the name of the
#facility, the name of the member formatted as a single column, and the cost.
#Order by descending cost, and do not use any subqueries.

q8 = '''
SELECT CONCAT_WS(' ', firstname, surname) AS Name, name AS Facility, CASE WHEN memid = 0 THEN slots * guestcost ELSE slots * membercost END AS Cost
FROM Bookings b
LEFT JOIN Facilities f USING (facid)
LEFT JOIN Members m USING (memid)
WHERE b.starttime LIKE '2012-09-14%' HAVING Cost > 30 
ORDER BY Cost DESC;
'''

In [10]:
# Q9: This time, produce the same result as in Q8, but using a subquery.

q9 = '''
SELECT sq.Name, sq.Facility, sq.Cost
FROM (
SELECT CONCAT_WS(' ', m.firstname, m.surname ) AS Name, f.name AS Facility, CASE WHEN m.memid = 0 THEN f.guestcost * b.slots ELSE f.membercost * b.slots END AS Cost
FROM Members m
LEFT JOIN Bookings b USING (memid)
LEFT JOIN Facilities f USING (facid)
WHERE b.starttime LIKE '2012-09-14%'
) AS sq
WHERE sq.Cost > 30
ORDER BY sq.Cost DESC;
'''

In [11]:
# Q10: Produce a list of facilities with a total revenue less than 1000.
#The output of facility name and total revenue, sorted by revenue. Remember
#that there's a different cost for guests and members!

q10 = '''
SELECT name, sum(revenue) as total_revenue 
FROM (SELECT f.name,  
       CASE WHEN b.memid > 0 THEN f.membercost * b.slots 
            ELSE f.guestcost * b.slots END AS revenue
FROM Facilities as f 
INNER JOIN Bookings as b
ON b.facid = f.facid) AS s
GROUP BY s.name
HAVING sum(revenue) < 1000
ORDER BY total_revenue;
'''

cursor.execute(q10)

print("{0:10s} \t {1:10s}".format('Facility', 'Total Revenue'))
for res in cursor.fetchall():
    print("{0:10s} \t {1:.2f}".format(*res))

Facility   	 Total Revenue
Table Tennis 	 180.00
Snooker Table 	 240.00
Pool Table 	 270.00


In [12]:
# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

q11 = '''
SELECT firstname, surname, (SELECT firstname FROM Members m2 WHERE m2.memid = m1.recommendedby AND m1.recommendedby > 0) AS rec_first, (SELECT surname FROM Members m2 WHERE m2.memid = m1.recommendedby AND m1.recommendedby > 0) AS rec_last
FROM Members m1
WHERE memid > 0
ORDER BY surname, firstname;
'''
cursor.execute(q11)

print("{0:10s} \t {1:10s} \t {2:10s} \t {3:10s}".format('First Name', 'Last Name', 'Rec First', 'Rec Last'))
for res in cursor.fetchall():
    try:
        print("{0:10s} \t {1:10s} \t {2:10s} \t {3:10s}".format(*res))
    except:
        print("{0:10s} \t {1:10s}".format(*res))

First Name 	 Last Name  	 Rec First  	 Rec Last  
Florence   	 Bader      	 Ponder     	 Stibbons  
Anne       	 Baker      	 Ponder     	 Stibbons  
Timothy    	 Baker      	 Jemima     	 Farrell   
Tim        	 Boothe     	 Tim        	 Rownam    
Gerald     	 Butters    	 Darren     	 Smith     
Joan       	 Coplin     	 Timothy    	 Baker     
Erica      	 Crumpet    	 Tracy      	 Smith     
Nancy      	 Dare       	 Janice     	 Joplette  
David      	 Farrell   
Jemima     	 Farrell   
Matthew    	 Genting    	 Gerald     	 Butters   
John       	 Hunt       	 Millicent  	 Purview   
David      	 Jones      	 Janice     	 Joplette  
Douglas    	 Jones      	 David      	 Jones     
Janice     	 Joplette   	 Darren     	 Smith     
Anna       	 Mackenzie  	 Darren     	 Smith     
Charles    	 Owen       	 Darren     	 Smith     
David      	 Pinker     	 Jemima     	 Farrell   
Millicent  	 Purview    	 Tracy      	 Smith     
Tim        	 Rownam    
Henrietta  	 Rumney     	 Ma

In [13]:
# Q12: Find the facilities with their usage by member, but not guests

q12 = '''
SELECT f.name, COUNT(*)
FROM Facilities f
INNER JOIN Bookings b
ON b.facid = f.facid
WHERE b.memid > 0
GROUP BY f.name;
'''

cursor.execute(q12)

print("{0:15s} \t {1:10s}".format('Facility', 'Usage'))
for res in cursor.fetchall():
    print("{0:15s} \t {1:.2f}".format(*res))

Facility        	 Usage     
Badminton Court 	 344.00
Massage Room 1  	 421.00
Massage Room 2  	 27.00
Pool Table      	 783.00
Snooker Table   	 421.00
Squash Court    	 195.00
Table Tennis    	 385.00
Tennis Court 1  	 308.00
Tennis Court 2  	 276.00


In [14]:
# Q13: Find the facilities usage by month, but not guests

q13 = '''
SELECT f.name, (strftime('%Y-%m', starttime)) AS Month, COUNT(*) AS Uses
FROM Facilities f
INNER JOIN Bookings b
ON b.facid = f.facid
WHERE b.memid > 0
GROUP BY f.name, Month;
'''

cursor.execute(q13)

print("{0:15s} \t {1:10s} \t {2:10s}".format('Facility', 'Month', 'Usage'))
for res in cursor.fetchall():
    print("{0:15s} \t {1:10s} \t {2:.2f}".format(*res))

Facility        	 Month      	 Usage     
Badminton Court 	 2012-07    	 51.00
Badminton Court 	 2012-08    	 132.00
Badminton Court 	 2012-09    	 161.00
Massage Room 1  	 2012-07    	 77.00
Massage Room 1  	 2012-08    	 153.00
Massage Room 1  	 2012-09    	 191.00
Massage Room 2  	 2012-07    	 4.00
Massage Room 2  	 2012-08    	 9.00
Massage Room 2  	 2012-09    	 14.00
Pool Table      	 2012-07    	 103.00
Pool Table      	 2012-08    	 272.00
Pool Table      	 2012-09    	 408.00
Snooker Table   	 2012-07    	 68.00
Snooker Table   	 2012-08    	 154.00
Snooker Table   	 2012-09    	 199.00
Squash Court    	 2012-07    	 23.00
Squash Court    	 2012-08    	 85.00
Squash Court    	 2012-09    	 87.00
Table Tennis    	 2012-07    	 48.00
Table Tennis    	 2012-08    	 143.00
Table Tennis    	 2012-09    	 194.00
Tennis Court 1  	 2012-07    	 65.00
Tennis Court 1  	 2012-08    	 111.00
Tennis Court 1  	 2012-09    	 132.00
Tennis Court 2  	 2012-07    	 41.00
Tennis Court 2  	 2012